In [29]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
import matplotlib.pyplot as plt
import torch.nn as nn
from PIL import Image, ImageDraw, ImageFont
import matplotlib.patches as patches
import random
import io
import utils
import preprocess as pre
import data_augmentation as aug
import DataLoader as dl

In [21]:
train_loader = dl.getCordTorchDatasetLoader("params_cord_initial.yaml", split='train')

Found cached dataset parquet (C:/Users/ipiri/.cache/huggingface/datasets/naver-clova-ix___parquet/naver-clova-ix--cord-v2-c97f979311033a44/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
C:\Users\ipiri\anaconda3\envs\ip\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ipiri\anaconda3\envs\ip\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [22]:
a = next(iter(train_loader))

In [26]:
a[0][0].shape

torch.Size([512, 512, 3])

In [28]:
class RegionProposalNetwork(nn.Module):
    def __init__(self, in_channels, out_channels, anchor_sizes, aspect_ratios, num_classes):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.anchor_sizes = anchor_sizes
        self.aspect_ratios = aspect_ratios
        self.num_classes = num_classes #number of classes including background class (0)
        
        # Defining RPN layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        
        self.bbox_head = nn.Conv2d(out_channels, len(anchor_sizes) * 4, kernel_size=1, stride=1, padding=0)
        self.conf_head = nn.Conv2d(out_channels, len(anchor_sizes) * num_classes, kernel_size=1, stride=1, padding=0)
    
        # Initialising Parameters for RPN layers
        self.initModelParams(self.conv1, 0, std=0.01)
        self.initModelParams(self.bbox_head, 0, std=0.01)
        self.initModelParams(self.conf_head, 0, std=0.01)
        
    def initModelParams(self, layer, mean, std):
        #Initial weights with respect to given mean and standard deviation
        nn.init.normal_(layer.weight, mean=mean, std=std)
        #Initial bias set to 0
        nn.init.constant_(layer.bias, 0)
    
    def forward(self, x):
        #Applying the ReLU function on the output of the conv1 layer (which is basically applying 3x3 conv on the given feature map
        x = nn.functional.relu(self.conv1)
        
        return self.bbox_head(x), self.conf_head(x)

In [ ]:
rpn = RegionProposalNetwork(512, 512, )